In [1]:
!pip install -q tensorboard
!pip install -q tensorflow-datasets

In [2]:
# Setup. Import libraries and load dataframes for Movielens data.
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import random

# run main function
path = "/storage/aliccp/"
log_path = "/storage/aliccp/"

df_train = pd.read_parquet(path+'/train_processed/')


print(len(df_train))

#filter out long tail products and reduce size of dataset
df_train['counter'] = 1
df_agg = df_train[['item_id','counter']].groupby('item_id').count()
df_agg = df_agg.sort_values("counter",ascending=False)[:10000]
df_train = pd.merge(df_agg, df_train, on=['item_id', 'item_id'], how='left')
print(len(df_train))
df_train['counter'] = 1
cust_agg = df_train[['user_id','counter']].groupby('user_id').count()
cust_agg = cust_agg.sort_values("counter",ascending=False)[:8000]
df_train = pd.merge(cust_agg, df_train, on=['user_id', 'user_id'], how='left')
print(len(df_train))

2023-03-11 18:18:24.257908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 18:18:25.017858: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 18:18:25.017934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 18:18:25.017944: W tensorfl

41999771
9477639


/tmp/ipykernel_2243/1411737331.py:28: FutureWarning: Passing 'suffixes' which cause duplicate columns {'counter_y'} in the result is deprecated and will raise a MergeError in a future version.
  df_train = pd.merge(cust_agg, df_train, on=['user_id', 'user_id'], how='left')


1473399


In [3]:
train = df_train[['item_id','user_id','user_categories', 'user_item_categories',
                                       'user_intentions','user_item_intentions','user_shops','user_item_brands','click']]

features = ['item_id','user_id','user_categories']
for i in features:
    train[i] = pd.to_numeric(train[i], errors='coerce',downcast="integer")
train= train.fillna(0)

/tmp/ipykernel_2243/257137575.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i] = pd.to_numeric(train[i], errors='coerce',downcast="integer")


In [5]:

hidden_units = (192,20)
embedding_size = 32

# Each instance will consist of two inputs: a single user id, and a single movie id
user_id_input = keras.Input(shape=(1,), name='user_id')
item_id_input = keras.Input(shape=(1,), name='item_id')
user_categories_input = keras.Input(shape=(1,), name='user_categories')
user_embedding = keras.layers.Embedding(len(train['user_id'])+1, embedding_size, 
                                       input_length=1, name='user_embedding')(user_id_input)
item_embedding = keras.layers.Embedding(len(train['item_id'])+1, embedding_size, 
                                        input_length=1, name='item_embedding')(item_id_input)
user_categories = keras.layers.Embedding(len(train['user_categories'])+1, embedding_size, 
                                        input_length=1, name='user_categories_embedding')(user_categories_input)
# Concatenate the embeddings (and remove the useless extra dimension)
concatenated = keras.layers.Concatenate()([user_embedding, item_embedding,user_categories])
out = keras.layers.Flatten()(concatenated)

# Add one or more hidden layers
for n_hidden in hidden_units:
    out = keras.layers.Dense(n_hidden, activation='relu')(out)

# A single output: our predicted rating
out = keras.layers.Dense(1, activation='sigmoid', name='prediction')(out)

model = keras.Model(
    inputs = [user_id_input, item_id_input, user_categories_input],
    outputs = out,
)
model.summary(line_length=88)

Model: "model"
________________________________________________________________________________________
 Layer (type)                Output Shape       Param #   Connected to                  
 user_id (InputLayer)        [(None, 1)]        0         []                            
                                                                                        
 item_id (InputLayer)        [(None, 1)]        0         []                            
                                                                                        
 user_categories (InputLayer  [(None, 1)]       0         []                            
 )                                                                                      
                                                                                        
 user_embedding (Embedding)  (None, 1, 32)      47148800  ['user_id[0][0]']             
                                                                                        
 item_

In [10]:
model.compile(
    tf.keras.optimizers.Adam(0.01),
    loss='MAE',
    metrics=[tf.keras.metrics.BinaryCrossentropy("BCE")],
)

In [11]:
model.fit(
    [train['user_id'], train['item_id'], train['user_categories']],
    train['click'],
    batch_size=5000,
    epochs=20)

Epoch 1/20
280/280 [==============================] - 41s 144ms/step - loss: 0.0348 - BCE: 2.4035 - val_loss: 0.0316 - val_BCE: 1.9224
Epoch 2/20
280/280 [==============================] - 12s 42ms/step - loss: 0.0306 - BCE: 2.5751 - val_loss: 0.0316 - val_BCE: 1.9218
Epoch 3/20
280/280 [==============================] - 9s 33ms/step - loss: 0.0306 - BCE: 2.5745 - val_loss: 0.0316 - val_BCE: 1.9216
Epoch 4/20
280/280 [==============================] - 8s 27ms/step - loss: 0.0306 - BCE: 2.5743 - val_loss: 0.0316 - val_BCE: 1.9214
Epoch 5/20
280/280 [==============================] - 8s 28ms/step - loss: 0.0306 - BCE: 2.5741 - val_loss: 0.0316 - val_BCE: 1.9213
Epoch 6/20
280/280 [==============================] - 7s 25ms/step - loss: 0.0306 - BCE: 2.5739 - val_loss: 0.0316 - val_BCE: 1.9213
Epoch 7/20
280/280 [==============================] - 7s 24ms/step - loss: 0.0306 - BCE: 2.5738 - val_loss: 0.0316 - val_BCE: 1.9212
Epoch 8/20
280/280 [==============================] - 7s 26ms/step